### Постановка задачи
Заполним отсутствующие значения по погоде интерполяционными данными.

Посчитаем модель линейной регрессии по первому зданию и найдем ее точность.

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [32]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### Загрузка данных

In [33]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")

### Отсечение здания 0 и отсутствующих значений

In [34]:
energy = energy[(energy["building_id"]==0)]

### Объединение данных

In [35]:
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "site_id", "floor_count"], axis=1)
del buildings
del weather
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 14 columns):
timestamp             8784 non-null object
building_id           8784 non-null int64
meter_reading         8784 non-null float64
primary_use           8784 non-null object
square_feet           8784 non-null int64
year_built            8784 non-null float64
floor_count           0 non-null float64
air_temperature       8781 non-null float64
cloud_coverage        4954 non-null float64
dew_temperature       8781 non-null float64
precip_depth_1_hr     8783 non-null float64
sea_level_pressure    8699 non-null float64
wind_direction        8534 non-null float64
wind_speed            8784 non-null float64
dtypes: float64(10), int64(2), object(2)
memory usage: 960.8+ KB
None


### Оптимизация памяти

In [36]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [37]:
energy = reduce_mem_usage(energy)

Потребление памяти меньше на 0.62 Мб (минус 66.1 %)


c:\users\nikolay\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  
c:\users\nikolay\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in less
  # Remove the CWD from sys.path while we load stuff.


### Интерполяция данных

In [38]:
energy["precip_depth_1_hr"] = energy["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed",
                       "precip_depth_1_hr", "sea_level_pressure"]
for col in interpolate_columns:
    energy[col] = energy[col].interpolate(limit_direction='both',
                            kind='cubic')

### Проверка качества интерполяции

In [39]:
pd.set_option('use_inf_as_na', True)
for col in interpolate_columns:
    print (col, "Inf+NaN:", energy[col].isnull().sum())

air_temperature Inf+NaN: 0
dew_temperature Inf+NaN: 0
cloud_coverage Inf+NaN: 0
wind_speed Inf+NaN: 0
precip_depth_1_hr Inf+NaN: 0
sea_level_pressure Inf+NaN: 0


### Разделение данных

In [40]:
energy_train, energy_test = train_test_split(energy[energy["meter_reading"]>0], test_size=0.2)
print (energy_train.head())

               timestamp  building_id  meter_reading primary_use  square_feet  \
4265 2016-06-26 17:00:00            0        232.125   Education         7432   
5267 2016-08-07 11:00:00            0        293.500   Education         7432   
4255 2016-06-26 07:00:00            0        242.250   Education         7432   
7387 2016-11-03 19:00:00            0        180.250   Education         7432   
4560 2016-07-09 00:00:00            0        288.000   Education         7432   

      year_built  floor_count  air_temperature  cloud_coverage  \
4265      2008.0          NaN        27.796875        4.000000   
5267      2008.0          NaN        25.000000        6.332031   
4255      2008.0          NaN        26.093750        5.000000   
7387      2008.0          NaN        28.906250        5.000000   
4560      2008.0          NaN        32.187500        6.000000   

      dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
4265        22.203125               

### Линейная регрессия

In [41]:
regression_columns = ["meter_reading", "air_temperature",
        "dew_temperature", "cloud_coverage", "wind_speed",
        "precip_depth_1_hr", "sea_level_pressure"]

energy_train_lr = pd.DataFrame(energy_train,
    columns=regression_columns)
y = energy_train_lr["meter_reading"]
x = energy_train_lr.drop(labels=["meter_reading"], axis=1)
model = LinearRegression().fit(x, y)
print (model.coef_, model.intercept_)

[ 2.57940166  3.66685087 -2.36935224 -1.97273319  0.13562709 -0.97385944] 1109.167249161545


### Предсказание и оценка модели

In [45]:
def calculate_model (x):
    lr = np.sum([x[col] * model.coef_[i] for i,col in enumerate(regression_columns[1:])])
    lr += model.intercept_
    x["meter_reading_lr_q"] = (np.log(1 + x.meter_reading) -
                               np.log(1 + lr))**2
    return x

energy_test = energy_test.apply(calculate_model,
                                    axis=1, result_type="expand")
energy_test_lr_rmsle = np.sqrt(energy_test["meter_reading_lr_q"].sum() / len(energy_test))
print ("Качество линейной регрессии:", energy_test_lr_rmsle, round(energy_test_lr_rmsle, 1))

Качество линейной регрессии: 0.19696713505628877 0.2
